In [ ]:
from os import getcwd
from os.path import abspath, join

from copy import deepcopy
from joblib import Parallel, delayed, parallel_backend
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from tqdm import tqdm

from src.data.load_dataset import generate_frequency_XOR
from src.models.estimator import RFClassifier, relu
from src.models.weights import sensilla_weights, classical_weights

In [ ]:
data_dir = abspath(join(getcwd(), '../../'))

#### Time-series of $0.1$s sampled at $2000$ Hz with $f_1=50$ Hz and $f_2=80$ Hz

In [ ]:
num_samples, sampling_rate, duration, freq1, freq2, snr, seed = 7000, 2000, 0.1, 50, 80, 0.8, 5
X, y = generate_frequency_XOR(num_samples, sampling_rate, freq1, freq2, duration, snr, seed=None, shuffle=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, stratify=y)

# params
num_neurons = sorted(set(np.logspace(0, 3, 50).astype('int')))
num_trials = 5
nonlinearity = relu
bias = 0
scale = 1
clf = LinearSVC(tol=1e-4, max_iter=1000)
n_jobs=6

#### Mechanosensory weights with $f_{lo}=50$Hz, $f_{hi}=90$Hz, and $\gamma=0.04$s

In [ ]:
kwargs = {'sampling_rate':sampling_rate, 'duration': duration, 'lowcut':50, 'highcut':90, 'decay_coef':0.04, 'scale': scale}
test_sensilla = {'hidden_size': [], 'mean': [], 'std_err': []}

for n in tqdm(num_neurons):
    # declare classifier, fit in parallel, and compute output score
    classifiers = [RFClassifier(n, sensilla_weights, bias, nonlinearity, deepcopy(clf), kwargs) for i in range(num_trials)]
    with parallel_backend('threading', n_jobs=n_jobs):
        Parallel()(delayed(RFclf.fit)(X_train, y_train) for RFclf in classifiers)
        test_accuracy = [RFclf.score(X_test, y_test) for RFclf in classifiers]
    test_sensilla['mean'].append(np.mean(test_accuracy))
    test_sensilla['std_err'].append(np.std(test_accuracy) / np.sqrt(num_trials))
    test_sensilla['hidden_size'].append(n)

#### Classical weights

In [ ]:
kwargs = {'scale':scale}
test_classical = {'hidden_size': [], 'mean': [], 'std_err': []}

for n in tqdm(num_neurons):
    # declare classifier, fit in parallel, and compute accuracy
    classifiers = [RFClassifier(n, classical_weights, bias, nonlinearity, deepcopy(clf), kwargs) for i in range(num_trials)]
    with parallel_backend('threading', n_jobs=n_jobs):
        Parallel()(delayed(RFclf.fit)(X_train, y_train) for RFclf in classifiers)
        test_accuracy = [RFclf.score(X_test, y_test) for RFclf in classifiers]
    test_classical['mean'].append(np.mean(test_accuracy))
    test_classical['std_err'].append(np.std(test_accuracy) / np.sqrt(num_trials))
    test_classical['hidden_size'].append(n)

#### Incompatible weights with $f_{lo}=10$Hz, $f_{hi}=60$Hz, and $\gamma=0.04$s

In [ ]:
kwargs = {'sampling_rate':sampling_rate, 'duration': duration, 'lowcut':10, 'highcut':60, 'decay_coef':0.04, 'scale': scale}
test_incompatible = {'hidden_size': [], 'mean': [], 'std_err': []}

for n in tqdm(num_neurons):
    # declare classifier, fit in parallel, and compute output score
    classifiers = [RFClassifier(n, sensilla_weights, bias, nonlinearity, deepcopy(clf), kwargs) for i in range(num_trials)]
    with parallel_backend('threading', n_jobs=n_jobs):
        Parallel()(delayed(RFclf.fit)(X_train, y_train) for RFclf in classifiers)
        test_accuracy = [RFclf.score(X_test, y_test) for RFclf in classifiers]
    test_incompatible['mean'].append(np.mean(test_accuracy))
    test_incompatible['std_err'].append(np.std(test_accuracy) / np.sqrt(num_trials))
    test_incompatible['hidden_size'].append(n)

In [ ]:
# save
test = {'sensilla': test_sensilla, 'classical': test_classical, 'incompatible': test_incompatible}
save_dir = data_dir + '/models/results/freq_XOR'
if not path.exists(save_dir):
    os.makedirs(save_dir)
with open(save_dir + '/freq_XOR_sensilla_estimator.pickle', 'wb') as handle:
    pickle.dump(test, handle, protocol=pickle.HIGHEST_PROTOCOL) 